# 7.5 批量规范化
- **目录**
  - 7.5.1 训练深层网络
  - 7.5.2 批量规范化层
    - 7.5.2.1 全连接层
    - 7.5.2.2 卷积层
    - 7.5.2.3 预测过程中的批量规范化
  - 7.5.3 从零实现
  - 7.5.4 使用批量规范化层的 LeNet
  - 7.5.5 简明实现
  - 7.5.6 争议

- 训练深层神经网络是十分困难的，特别是在较短的时间内使他们收敛更加棘手。
- **批量规范化（batch normalization）**，这是一种流行且有效的技术，**可持续加速深层网络的收敛速度。**
- 再结合在7.6节中将介绍的残差块，批量规范化使得研究人员能够训练**100层以上**的网络。

## 7.5.1 训练深层网络

为什么需要批量规范化层呢？让我们来回顾一下训练神经网络时出现的一些**实际挑战**。

- 首先，**数据预处理**的方式通常会对最终结果产生巨大影响。
回想一下我们应用多层感知机来预测房价的例子（4.10节）。
使用真实数据时，我们的第一步是**标准化输入特征**，使其平均值为0，方差为1。
直观地说，这种标准化可以很好地与我们的优化器配合使用，因为它可以**将参数的量级进行统一。**

- 第二，对于典型的多层感知机或卷积神经网络。当我们训练时，中间层中的变量（例如，多层感知机中的仿射变换输出）可能具有更广的变化范围：不论是沿着从输入到输出的层，跨同一层中的单元，或是随着时间的推移，模型参数的随着训练**更新变幻莫测**。
批量规范化的发明者非正式地假设，这些变量分布中的这种偏移可能会**阻碍网络的收敛**。
直观地说，我们可能会猜想，如果一个层的可变值（即激活值）是另一层的100倍，这可能需要**对学习率进行补偿调整**。

- 第三，**更深层的网络很复杂，容易过拟合。** 这意味着正则化变得更加重要。

批量规范化应用于单个可选层（也可以应用到所有层），其原理如下：在每次训练迭代中，我们首先规范化输入，即**通过减去其均值并除以其标准差，其中两者均基于当前小批量处理**。（比如样本批量大小为256，那么在这个256个样本的范围内对数据进行规范化）。
正是由于这个基于**批量统计**的**标准化**，才有了**批量规范化**的名称。

请注意，如果我们尝试使用大小为1的小批量应用批量规范化，我们将无法学到任何东西。
这是因为在减去均值之后，每个隐藏单元将为0。
所以，**只有使用足够大的小批量，批量规范化这种方法才是有效且稳定的**。
请注意，在应用批量规范化时，批量大小的选择可能比没有批量规范化时更重要。

从形式上来说，用$\mathbf{x} \in \mathcal{B}$表示一个来自小批量$\mathcal{B}$的输入，批量规范化$\mathrm{BN}$根据以下表达式转换$\mathbf{x}$：

$$\mathrm{BN}(\mathbf{x}) = \boldsymbol{\gamma} \odot \frac{\mathbf{x} - \hat{\boldsymbol{\mu}}_\mathcal{B}}{\hat{\boldsymbol{\sigma}}_\mathcal{B}} + \boldsymbol{\beta}.\tag {7.5.1}$$


在公式7.5.1中，$\hat{\boldsymbol{\mu}}_\mathcal{B}$是小批量$\mathcal{B}$的样本均值，$\hat{\boldsymbol{\sigma}}_\mathcal{B}$是小批量$\mathcal{B}$的样本标准差。
应用标准化后，生成的小批量的平均值为0和单位方差为1。

由于单位方差（与其他一些魔法数）是一个主观的选择，因此我们通常包含
**拉伸参数（scale）**$\boldsymbol{\gamma}$和**偏移参数（shift）**$\boldsymbol{\beta}$，它们的形状与$\mathbf{x}$相同。
**请注意，$\boldsymbol{\gamma}$和$\boldsymbol{\beta}$是需要与其他模型参数一起学习的参数**。

由于在训练过程中，中间层的变化幅度不能过于剧烈，而批量规范化将每一层主动居中，并将它们重新调整为给定的平均值和大小（通过$\hat{\boldsymbol{\mu}}_\mathcal{B}$和${\hat{\boldsymbol{\sigma}}_\mathcal{B}}$）。

从形式上来看，我们计算出公式7.5.1中的$\hat{\boldsymbol{\mu}}_\mathcal{B}$和${\hat{\boldsymbol{\sigma}}_\mathcal{B}}$，如下所示：

$$\begin{aligned} \hat{\boldsymbol{\mu}}_\mathcal{B} &= \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} \mathbf{x},\\
\hat{\boldsymbol{\sigma}}_\mathcal{B}^2 &= \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} (\mathbf{x} - \hat{\boldsymbol{\mu}}_{\mathcal{B}})^2 + \epsilon.\end{aligned} \tag{7.5.2}$$

请注意，**我们在方差估计值中添加一个小的常量$\epsilon > 0$，以确保我们永远不会尝试除以零**，即使在经验方差估计值可能消失的情况下也是如此。估计值$\hat{\boldsymbol{\mu}}_\mathcal{B}$和${\hat{\boldsymbol{\sigma}}_\mathcal{B}}$通过使用平均值和方差的噪声（noise）估计来抵消缩放问题。
你可能会认为这种噪声是一个问题，而事实上它是有益的。

事实证明，这是深度学习中一个反复出现的主题。
由于尚未在理论上明确的原因，**优化中的各种噪声源通常会导致更快的训练和较少的过拟合**：这种变化似乎是正则化的一种形式。
在一些初步研究中，将批量规范化的性质与贝叶斯先验相关联。这些理论揭示了为什么批量规范化最适应$50 \sim 100$范围中的中等批量大小的难题。

另外，批量规范化层在”训练模式“（通过小批量统计数据规范化）和“预测模式”（通过数据集统计规范化）中的功能不同。
- 在训练过程中，我们无法得知使用整个数据集来估计平均值和方差，所以**只能根据每个小批次的平均值和方差不断训练模型**。
- 在预测模式下，可以**根据整个数据集**精确计算批量规范化所需的平均值和方差。

现在，我们了解一下批量规范化在实践中是如何工作的。

- **要点：**
  - 批量规范化（Batch Normalization，BN）的产生主要是为了解决深度神经网络训练过程中的一些挑战，包括**数据预处理方式对结果的影响**，**隐藏层变量的广泛变化**以及**深度网络的复杂性和过拟合问题**。
  - 批量规范化通过在每次训练迭代中对输入进行标准化（即，减去其均值并除以其标准差，这两者都基于当前的小批量）来工作。然后应用比例系数和比例偏移。这**需要使用足够大的小批量**才能使该方法有效且稳定。
  - 批量规范化的操作可以公式化表示为：$\mathrm{BN}(\mathbf{x}) = \boldsymbol{\gamma} \odot \frac{\mathbf{x} - \hat{\boldsymbol{\mu}}_\mathcal{B}}{\hat{\boldsymbol{\sigma}}_\mathcal{B}} + \boldsymbol{\beta}$，其中$\hat{\boldsymbol{\mu}}_\mathcal{B}$和$\hat{\boldsymbol{\sigma}}_\mathcal{B}$分别是小批量的样本均值和标准差， $\boldsymbol{\gamma}$ 和$\boldsymbol{\beta}$ 是可学习的拉伸参数和偏移参数。
  - 批量规范化将**每一层主动居中，并将它们重新调整为给定的平均值和大小**，因此在训练过程中，**中间层的变化幅度不能过于剧烈**。具体解释如下：
    - 批量规范化通过调整每一层的激活值，使其符合预设的均值和方差，从而避免了所谓的内部协变量转移问题，即训练过程中每一层输入分布的改变。
    - 假设我们正在训练一个深度神经网络，其中有一个隐藏层的激活函数是ReLU。在训练初期，这个隐藏层的输入可能在零附近集中，使得ReLU函数的输出也在零附近。然而，随着训练的进行，由于前面层参数的更新，这个隐藏层的输入可能逐渐偏向正无穷，导致ReLU函数的输出也趋向正无穷。这样，后面的层就需要去适应这一变化，这会降低训练速度并可能导致不稳定性。
    - 如果我们在这个隐藏层之后加入批量规范化层，那么无论其输入如何变化，批量规范化都会将其调整为预设的均值和方差（例如，均值为0，方差为1），这样ReLU函数的输出便会始终保持在相同的范围内。这就保证了中间层的变化幅度不能过于剧烈，有助于提升模型训练的稳定性和效率。
    
  - 在优化中的各种噪声源通常会导致更快的训练和较少的过拟合，这是深度学习中的一个反复出现的主题。
  - 批量规范化在训练模式和预测模式下的功能是不同的。训练时我们使用**每个批量的均值和方差**，而预测时我们使用**整个数据集的精确均值和方差**。

## 7.5.2 批量规范化层
- 批量规范化和其他层之间的一个关键区别是，由于批量规范化在完整的小批量上运行，因此我们不能像以前在引入其他层时那样忽略批量大小。

- 全连接层和卷积层的**批量规范化实现略有不同**。

### 7.5.2.1 全连接层

- 通常，我们将批量规范化层置于**全连接层中的仿射变换和激活函数之间**。
- 设全连接层的输入为u，权重参数和偏置参数分别为$\mathbf{W}$和$\mathbf{b}$，激活函数为$\phi$，批量规范化的运算符为$\mathrm{BN}$。
- 使用批量规范化的全连接层的输出的计算详情如下：
$$\mathbf{h} = \phi(\mathrm{BN}(\mathbf{W}\mathbf{x} + \mathbf{b}) ). \tag{7.5.3}$$
回想一下，均值和方差是在应用变换的"相同"小批量上计算的。

### 7.5.2.2 卷积层

- 同样，对于卷积层，我们可以在**卷积层之后和非线性激活函数之前**应用批量规范化。
- 当卷积有多个输出通道时，我们需要**对这些通道的“每个”输出执行批量规范化，每个通道都有自己的拉伸（scale）和偏移（shift）参数，这两个参数都是标量**。
- 假设我们的小批量包含$m$个样本，并且对于每个通道，卷积的输出具有高度$p$和宽度$q$。
- 那么对于卷积层，我们在每个输出通道的$m \cdot p \cdot q$个元素上同时执行每个批量规范化。
- 因此，在计算平均值和方差时，我们会收集所有空间位置的值，然后在给定通道内应用相同的均值和方差，以便在每个空间位置对值进行规范化。

-----------------
- **说明：拉伸与偏移参数的作用**
  - **恢复表达能力**
    - **标准化的影响**：
      - 在标准化过程中，每个通道的输出都被转换为零均值和单位方差，这**虽然有助于消除输入数据的分布不稳定性，但也可能导致模型丧失部分表达能力**。
      - 如果没有拉伸和偏移参数，标准化后的输出将严格限制在标准正态分布（均值为0，方差为1）内，这**可能会限制模型学习到复杂的非线性变换**。
    - **恢复原始分布或调整分布**：
      - 拉伸参数 $\gamma$ 和偏移参数 $\beta$ 允许模型重新调整每个通道的输出分布，这使得模型即使在标准化之后，也能保持足够的表达能力。
      - 例如，$\gamma$ 可以调整标准化后的输出的方差，而 $\beta$ 可以调整标准化后的输出的均值。
      - 通过学习这两个参数，模型可以在需要时恢复到原始的分布或调整到更适合当前任务的分布。
  - **稳定训练过程**
    - **对抗梯度消失和梯度爆炸**：
      - 由于标准化操作，**所有层的输入数据分布保持稳定，这有助于缓解深度网络中的梯度消失和梯度爆炸问题**。
      - 但是，如果没有 $\gamma$ 和 $\beta$，模型可能无法有效地利用标准化后的输出。
      - **拉伸和偏移参数确保了模型在标准化的基础上仍然可以学习到丰富的特征**，这对提高模型的训练稳定性至关重要。
  - **增强模型的灵活性**
    - **学习复杂的非线性变换**：
      - 在卷积层的输出经过激活函数之前，批量规范化确保了输入到激活函数的值在一个稳定的范围内。通过添加可学习的拉伸参数 $\gamma$ 和偏移参数 $\beta$，批量规范化不仅标准化了输入，还能够**为每个通道引入进一步的调整**。
      - 这意味着模型可以**更灵活地适应数据**，学习到更复杂的非线性变换。尤其是在深度神经网络中，这种灵活性可以显著提高模型的表达能力和性能。
    - **与激活函数的交互**：
      - 激活函数（如 ReLU、Sigmoid、Tanh 等）在整个神经网络中起着至关重要的作用，主要目的和功能在于引入了非线性，使得神经网络能够学习复杂的函数映射。
      - 批量规范化会在激活函数之前应用，这意味着激活函数接收到的输入已经过标准化并且线性变换过。这种标准化和线性变换确保了**激活函数在一个更适合的范围内工作**，从而避免了梯度消失或梯度爆炸的现象。
      - 例如，对于 ReLU 激活函数，如果输入的分布经过标准化，并且拉伸参数 $\gamma$ 的值适当，输入值更有可能落在 ReLU 的有效区间（即 $x > 0$），从而减少“**死区**”现象（即大量神经元的输出为 0，导致它们在反向传播中不再更新）。
-------------

### 7.5.2.3 预测过程中的批量规范化

正如我们前面提到的，批量规范化在训练模式和预测模式下的行为通常不同。
- 首先，将训练好的模型用于预测时，我们不再需要样本均值中的噪声以及在小批量上估计的样本方差了。
- 其次，可能需要使用模型对逐个样本进行预测。
- 一种常用的方法是通过**移动平均**估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。
- 和暂退法一样，**批量规范化层在训练模式和预测模式下的计算结果也是不一样的**。
 

## 7.5.3 从零实现

- 从头开始实现一个具有张量的批量规范化层。

In [25]:
%matplotlib inline
import torch
from torch import nn
from d2l import torch as d2l


def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 通过is_grad_enabled来判断当前模式是训练模式还是预测模式
    if not torch.is_grad_enabled():
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差        
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            '''            
            这里的特征维如何理解？
            比如某小批量样本数据的形状为：(256，8)            
            比如此处数据形状为256行8列，每行是一条样本数据，8个特征，
            那么在dim=0上求均值，就是在列上求均值，也就是每个特征的、256条数据的均值；
            标准差也是如此处理。
            因此均值和标准差都是形状为(8, )的向量。
            '''
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。
            # 这里我们需要保持X的形状以便后面可以做广播运算
            '''
            假设X形状为(256, 3, 96, 96)，那么就是将X按照通道分成3份。
            可以如此想象：
            （1）256张图片，高和宽都是96像素，每张图片有红绿蓝三个通道。
            （2）将256张图片的红色通道的取出合并成(256, 96, 96)矩阵。
            （3）对此256*96*96个数据求取均值和方差。
            （4）然后依次对绿色和蓝色通道同样如此处理。
            （5）最后得到3个通道的均值。
            （6）由于keepdim=True，因此将均值和方差保存在通道维（即第2维），作为4阶张量输出
                其形状皆为：(1, 3, 1, 1)。
            '''
            mean = X.mean(dim=(0, 2, 3), keepdim=True)
            var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
        # 训练模式下，用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        ## 其实就是新老平均数的加权平均，新老方差也是如此处理
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
        
    ## 缩放和移位与X_hat相乘都是按元素进行计算，所以二者必须X_hat的形状一致，
    ## 包括维度也要一致。
    Y = gamma * X_hat + beta  # 缩放和移位
    return Y, moving_mean.data, moving_var.data

--------------------

- **说明：批量规范化计算**

In [26]:
'''
样本数据的形状为(5, 3, 8, 8)，即5张3通道高8宽8的图片。
注意：在卷积层的计算过程中，规范化通道不一定是彩色图片的3个通道，
相反一般是卷积核的通道数，通俗点理解就是有多少个卷积核。
'''
import numpy as np
t= 3.+6.*torch.randn(960)
t=t.reshape((5,3,8,8))
##通道均值与标准差
t.mean(dim=(0,2,3),keepdim=True),t.std(dim=(0,2,3),keepdim=True),\
t.mean(dim=(0,2,3),keepdim=True).shape,t.std(dim=(0,2,3),keepdim=True).shape

(tensor([[[[2.8348]],
 
          [[3.7608]],
 
          [[3.2927]]]]),
 tensor([[[[6.0571]],
 
          [[5.6249]],
 
          [[6.0935]]]]),
 torch.Size([1, 3, 1, 1]),
 torch.Size([1, 3, 1, 1]))

In [27]:
'''
X.shape等于2的情况。
t1的形状为(256,8)，
'''
t1 = 3.0 + 6.0 * torch.randn(256*8).reshape(256, 8)
t1.shape,t1.mean(dim=0),t1.std(dim=0),t1.mean(dim=0).shape,t1.std(dim=0).shape

(torch.Size([256, 8]),
 tensor([2.5931, 2.6385, 3.0882, 3.1635, 3.4966, 3.7967, 2.6759, 2.7216]),
 tensor([5.9543, 5.7969, 5.5683, 6.4073, 6.0013, 5.7110, 5.7822, 5.8127]),
 torch.Size([8]),
 torch.Size([8]))

-------------------

- 现在可以**创建一个正确的`BatchNorm`层**。
  - 这个层将保持适当的参数：**拉伸`gamma`和偏移`beta`,这两个参数将在训练过程中更新**。
  - 此外，我们的层将保存均值和方差的移动平均值，以便在模型预测期间随后使用。

- 撇开算法细节，注意实现层的基础设计模式。
  - 通常情况下，我们**用一个单独的函数定义其数学原理**，比如说`batch_norm`。
  - 然后，我们将此功能**集成到一个自定义层**中，其代码主要处理数据移动到训练设备（如GPU）、分配和初始化任何必需的变量、跟踪移动平均线（此处为均值和方差）等问题。
- 为了方便起见，我们并不担心在这里自动推断输入形状，因此我们需要指定整个特征的数量。
- 深度学习框架中的批量规范化API将解决上述问题，稍后将展示这一点。


In [28]:
class BatchNorm(nn.Module):
    # num_features：完全连接层的输出特征数或卷积层的输出通道数。
    ##  num_dims：2表示完全连接层，4表示卷积层
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0
        ##这两个参数的形状必须与规范化输出的形状一致
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 非模型参数的变量初始化为0和1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var
        # 复制到X所在显存上
        ## 在使用GPU进行计算时很重要
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        '''
        保存更新过的moving_mean和moving_var，
        调用前面的批量规范化函数。
        移动平均其实就是使用老平均数与新的批量平均数的加权平均，
        前者的权重为0.9，后者为0.1。
        所谓的新老移动平均与方差就是前、后批量之间的移动平均与方差。
        '''
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

##  7.5.4 使用批量规范化层的 LeNet

- 下面将`BatchNorm`应用于**LeNet模型**（ 6.6节）。
- 注意：批量规范化放在卷积层或全连接层之后、相应的激活函数之前。


In [29]:
##BatchNorm直接作为一个层放在卷积层或全连接层与激活函数之间
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5), BatchNorm(6, num_dims=4), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), BatchNorm(16, num_dims=4), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
    nn.Linear(16*4*4, 120), BatchNorm(120, num_dims=2), nn.Sigmoid(),
    nn.Linear(120, 84), BatchNorm(84, num_dims=2), nn.Sigmoid(),
    nn.Linear(84, 10))

- 和前面一样，仍在Fashion-MNIST数据集上训练网络。
- 这个代码与我们第一次训练LeNet（6.6节）时几乎完全相同，主要区别在于**学习率大得多**。


In [31]:
import time
import numpy as np
lr, num_epochs, batch_size = 1.0, 10, 256
start = time.time()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())
end = time.time()
print(f'3090ti上模型训练耗时：{int(np.floor((end-start)/60))}分钟{(int(end-start)%60) }秒')

<img src = '..\img\7_5_1.png' height=350 width=350>

- 看看从第一个批量规范化层中学到的`gamma`和偏移参数`beta`。


In [ ]:
net[1].gamma.reshape((-1,)), net[1].beta.reshape((-1,))

## 7.5.5 简明实现

- 也可以直接使用深度学习框架Pytorch中定义的`BatchNorm2d`。

In [ ]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5), nn.BatchNorm2d(6), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.BatchNorm2d(16), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
    nn.Linear(256, 120), nn.BatchNorm1d(120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.BatchNorm1d(84), nn.Sigmoid(),
    nn.Linear(84, 10))

- 使用相同超参数来训练模型
- 通常高级API变体运行速度快得多，因为它的代码已编译为C++或CUDA，而我们的自定义代码由Python实现，速度会慢很多。


In [ ]:
## 原参数下，仍无法在MX250上运行
import time
import numpy as np
lr, num_epochs, batch_size = 1.0, 10, 256
start = time.time()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())
end = time.time()
print(f'3090ti上模型训练耗时：{int(np.floor((end-start)/60))}分钟{(int(end-start)%60) }秒')

<img src = '..\img\7_5_2.png' height=350 width=350>

- **训练速度比非批量规范化训练方式要快很多倍**

## 7.5.6 争议

直观地说，批量规范化被认为可以使优化更加平滑。
然而，我们必须小心区分直觉和对我们观察到的现象的真实解释。
回想一下，我们甚至不知道简单的神经网络（多层感知机和传统的卷积神经网络）为什么如此有效。
即使在暂退法和权重衰减的情况下，它们仍然非常灵活，因此无法通过常规的学习理论泛化保证来解释它们是否能够泛化到看不见的数据。

在提出批量规范化的论文中(Sergey Ioffe，Christian Szegedy. Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift)，作者除了介绍了其应用，还解释了其原理：通过减少**内部协变量偏移（internal covariate shift）**。
据推测，作者所说的“内部协变量转移”类似于上述的投机直觉，即变量值的分布在训练过程中会发生变化。
然而，这种解释有两个问题：
- 1、这种偏移与严格定义的**协变量偏移（covariate shift）** 非常不同，所以这个名字用词不当。
- 2、这种解释只提供了一种不明确的直觉，但留下了一个有待后续挖掘的问题：为什么这项技术如此有效？

本书旨在传达实践者用来发展深层神经网络的直觉。
然而，重要的是将这些指导性直觉与既定的科学事实区分开来。
最终，当你掌握了这些方法，并开始撰写自己的研究论文时，你会希望清楚地区分技术和直觉。

批量规范化已经被证明是一种不可或缺的方法。它适用于几乎所有图像分类器，并在学术界获得了数万引用。

- **要点：**
  - 批量规范化被认为可以使优化过程更加平滑，但其**真正的工作机制并不完全清楚**。
  - 简单神经网络（如多层感知机和传统卷积神经网络）的效用超出了常规学习理论泛化保证的解释范围。
  - 批量规范化的原始论文提出了一种“**减少内部协变量偏移**”的理论来解释它的工作原理，但这种理论有其问题：名称可能误导人，且只提供了模糊的  直觉，没有明确解释为什么这项技术有效。
  - **在实践中，批量规范化已经被证明是一种非常有效和广泛应用的方法，尤其在图像分类任务中效果显著**。
  - 对于深度学习从业者和研究人员，重要的是能够区分实践中的**直觉**和**科学事实**，并在撰写研究论文时**明确区分技术和直觉**。

-----------------------
- **说明：如何理解“简单神经网络（如多层感知机和传统卷积神经网络）的效用超出了常规学习理论泛化保证的解释范围”这句话？**
  - 这里的“简单神经网络（如多层感知机和传统卷积神经网络）的效用超出了常规学习理论泛化保证的解释范围”是指，现在的理论理解还无法完全解释为什么这些网络在实践中表现得如此优秀。
  - 首先，让来理解一下学习理论和泛化。在机器学习中，"学习"通常被定义为一个系统的能力，通过从数据中学习，以改善其在未见过的数据上的性能，这种能力叫做泛化。更具体地说，当训练一个模型时，希望它不仅在训练数据上表现良好，而且在从未见过的测试数据上也有良好的表现。
  - 然而，理论上，预计模型的泛化能力应该与其复杂性成反比 —— 模型越复杂（即自由参数更多），它就越有可能对训练数据过拟合，也就是记住了训练数据的特定噪声和特性，而无法很好地泛化到新的数据上。这就是所谓的"overfitting"问题。
  - 但是，在深度学习中，经常看到相对于其参数的数量，神经网络似乎具有出奇的泛化能力。例如，可以训练一个拥有数百万甚至数十亿参数的神经网络，并且如果采取适当的正则化策略（如dropout、权重衰减等），这些网络通常可以在未见过的数据上表现良好。这是违反了的直觉的，因为根据的理论理解，这样大规模的网络应该容易过拟合，但实际上并非如此。
  - 这就是为什么说神经网络的效用超出了常规学习理论泛化保证的解释范围。也就是说，的理论理解还没有赶上实践中的观察结果，这是当前深度学习研究领域的一个重要挑战和研究方向。
--------------------

## 小结

* 在模型训练过程中，批量规范化利用批量的均值和标准差，不断调整神经网络的中间输出，使整个神经网络各层的中间输出值更加稳定。
* 批量规范化在全连接层和卷积层的使用略有不同。
* 批量规范化层和暂退层一样，在训练模式和预测模式下计算不同。
* 批量规范化有许多有益的副作用，主要是正则化。另一方面，”减少内部协变量偏移“的原始动机似乎不是一个有效的解释。